# GPX Reader - GPS Exchange Format

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://logiqx.github.io/gps-wizard/) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with Foobar. If not, see <https://www.gnu.org/licenses/>.

In [1]:
import os
import sys
import time

from lxml import etree
from dateutil.parser import parse

import numpy as np

import unittest

from base_reader import BaseReader

## Main Class

In [2]:
class GpxReader(BaseReader):
    '''GPX file - GPS Exchange Format'''

    def __init__(self, filename):
        '''Basic init just records the filename'''

        super().__init__(filename)


    def load(self):
        '''Load file into memory'''

        # Parse GPX file using lxml
        tree = etree.parse(self.filename)
        root = tree.getroot()
        
        # Create namespace mappings
        nsmap = {k if k is not None else 'default':v for k,v in root.nsmap.items()}

        # Pre-construct expected tags
        ns = '{' + nsmap['default'] + '}'
        eleTag = ns + 'ele'
        timeTag = ns + 'time'
        courseTag = ns + 'course'
        cogTag = ns + 'cog'
        speedTag = ns + 'speed'
        satTag = ns + 'sat'
        hdopTag = ns + 'hdop'

        # Use the first track only
        trks = root.xpath(".//default:trk", namespaces=nsmap)
        trk = trks[0]
        
        # Save the track name
        names = trk.xpath(".//default:name", namespaces=nsmap)
        if names:
            self.name = names[0].text

        # Find all trackpoints
        trkpts = trk.xpath(".//default:trkpt", namespaces=nsmap)
        self.numRecords = len(trkpts)

        # Create empty ndarrays
        lat = np.zeros(self.numRecords, dtype='float64')
        lon = np.zeros(self.numRecords, dtype='float64')
        ele = np.zeros(self.numRecords, dtype='float64')
        ts = np.zeros(self.numRecords, dtype='float64')
        cog = np.zeros(self.numRecords, dtype='float32')
        sog = np.zeros(self.numRecords, dtype='float32')
        sat = np.zeros(self.numRecords, dtype='uint8')
        hdop = np.zeros(self.numRecords, dtype='float32')

        # Process individual trackpoints
        for i in range(self.numRecords):
            lat[i] = trkpts[i].attrib['lat']
            lon[i] = trkpts[i].attrib['lon']

            for child in trkpts[i].getchildren():
                if child.text is not None:
                    if child.tag == eleTag:
                        ele[i] = child.text
                    elif child.tag == timeTag:
                        ts[i] = parse(child.text).timestamp()
                    elif child.tag == courseTag or child.tag == cogTag:
                        cog[i] = child.text
                    elif child.tag == speedTag:
                        sog[i] = child.text
                    elif child.tag == satTag:
                        sat[i] = child.text
                    elif child.tag == hdopTag:
                        hdop[i] = child.text

        # Retain populated arrays
        if np.count_nonzero(lat) > 0:
            self.data['lat'] = np.round(lat, 7)
        if np.count_nonzero(lon) > 0:
            self.data['lon'] = np.round(lon, 7)
        if np.count_nonzero(ele) > 0:
            self.data['ele'] = np.round(ele, 3)
        if np.count_nonzero(ts) > 0:
            self.data['ts'] = np.round(ts, 3)
        if np.count_nonzero(cog) > 0:
            self.data['cog'] = np.round(cog, 3)
        if np.count_nonzero(sog) > 0:
            self.data['sog'] = np.round(sog, 3)
        if np.count_nonzero(sat) > 0:
            self.data['sat'] = sat
        if np.count_nonzero(hdop) > 0:
            self.data['hdop'] = np.round(hdop, 2)

## Unit Tests

In [3]:
class TestCorosData(unittest.TestCase):
    '''Class to test COROS data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(corosReader.numRecords, 3809)

        for fieldName in corosReader.data:
            self.assertEqual(corosReader.data[fieldName].size, 3809)


    def testHdop(self):
        '''Test the horizontal dilution of precision is as expected'''

        self.assertEqual(corosReader.data['hdop'].min(), np.float32(0.7))
        self.assertEqual(corosReader.data['hdop'].max(), np.float32(1.6))


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(corosReader.data['sat'].min(), np.uint8(0))
        self.assertEqual(corosReader.data['sat'].max(), np.uint8(14))


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(corosReader.data['ts'].min(), np.float64(1649672017.0))
        self.assertEqual(corosReader.data['ts'].max(), np.float64(1649677777.0))


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(corosReader.data['lat'].min(), np.float64(50.5705683))
        self.assertEqual(corosReader.data['lat'].max(), np.float64(50.5832829))


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(corosReader.data['lon'].min(), np.float64(-2.4620381))
        self.assertEqual(corosReader.data['lon'].max(), np.float64(-2.4559603))


    def testCog(self):
        '''Test the course over ground is as expected'''

        self.assertEqual(corosReader.data['cog'].min(), np.float32(0))
        self.assertEqual(corosReader.data['cog'].max(), np.float32(359))

In [4]:
class TestSbnData(unittest.TestCase):
    '''Class to test GT-31 SBN data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(sbnReader.numRecords, 4161)

        for fieldName in sbnReader.data:
            self.assertEqual(sbnReader.data[fieldName].size, 4161)


    def testHdop(self):
        '''Test the horizontal dilution of precision is as expected'''

        self.assertEqual(sbnReader.data['hdop'].min(), np.float32(0.8))
        self.assertEqual(sbnReader.data['hdop'].max(), np.float32(2.0))


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(sbnReader.data['sat'].min(), np.uint8(4))
        self.assertEqual(sbnReader.data['sat'].max(), np.uint8(10))


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(sbnReader.data['ts'].min(), np.float64(1649672182.0))
        self.assertEqual(sbnReader.data['ts'].max(), np.float64(1649678792.0))


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(sbnReader.data['lat'].min(), np.float64(50.5710160))
        self.assertEqual(sbnReader.data['lat'].max(), np.float64(50.5833319))


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(sbnReader.data['lon'].min(), np.float64(-2.4620455))
        self.assertEqual(sbnReader.data['lon'].max(), np.float64(-2.4563038))


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(sbnReader.data['ele'].min(), np.float64(-3.02))
        self.assertEqual(sbnReader.data['ele'].max(), np.float64(11.93))


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(sbnReader.data['sog'].min(), np.float32(0.13))
        self.assertEqual(sbnReader.data['sog'].max(), np.float32(16.83))


    def testCog(self):
        '''Test the course over ground is as expected'''

        self.assertEqual(sbnReader.data['cog'].min(), np.float32(0.01))
        self.assertEqual(sbnReader.data['cog'].max(), np.float32(359.92))

In [5]:
class TestSbpData(unittest.TestCase):
    '''Class to test GT-31 SBP data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(sbpReader.numRecords, 4162)

        for fieldName in sbpReader.data:
            self.assertEqual(sbpReader.data[fieldName].size, 4162)


    def testHdop(self):
        '''Test the horizontal dilution of precision is as expected'''

        self.assertEqual(sbpReader.data['hdop'].min(), np.float32(0.8))
        self.assertEqual(sbpReader.data['hdop'].max(), np.float32(2.0))


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(sbpReader.data['sat'].min(), np.uint8(0))
        self.assertEqual(sbpReader.data['sat'].max(), np.uint8(10))


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(sbpReader.data['ts'].min(), np.float64(1649672161.0))
        self.assertEqual(sbpReader.data['ts'].max(), np.float64(1649678793.0))


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(sbpReader.data['lat'].min(), np.float64(50.5710156))
        self.assertEqual(sbpReader.data['lat'].max(), np.float64(50.5833410))


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(sbpReader.data['lon'].min(), np.float64(-2.4620455))
        self.assertEqual(sbpReader.data['lon'].max(), np.float64(-2.4563038))


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(sbpReader.data['ele'].min(), np.float64(-3.06))
        self.assertEqual(sbpReader.data['ele'].max(), np.float64(11.93))


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(sbpReader.data['sog'].min(), np.float32(0.01))
        self.assertEqual(sbpReader.data['sog'].max(), np.float32(16.83))


    def testCog(self):
        '''Test the course over ground is as expected'''

        self.assertEqual(sbpReader.data['cog'].min(), np.float32(0.01))
        self.assertEqual(sbpReader.data['cog'].max(), np.float32(359.92))

In [6]:
class TestFenix6Data(unittest.TestCase):
    '''Class to test Garmin Fenix 6 data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(fenix6Reader.numRecords, 1012)

        for fieldName in fenix6Reader.data:
            self.assertEqual(fenix6Reader.data[fieldName].size, 1012)


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(fenix6Reader.data['ts'].min(), np.float64(1650638705.0))
        self.assertEqual(fenix6Reader.data['ts'].max(), np.float64(1650644048.0))


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(fenix6Reader.data['lat'].min(), np.float64(50.8089770))
        self.assertEqual(fenix6Reader.data['lat'].max(), np.float64(50.8154146))


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(fenix6Reader.data['lon'].min(), np.float64(-0.9091131))
        self.assertEqual(fenix6Reader.data['lon'].max(), np.float64(-0.9017256))


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(fenix6Reader.data['ele'].min(), np.float64(-20.2))
        self.assertEqual(fenix6Reader.data['ele'].max(), np.float64(24.6))

In [7]:
class TestOsData(unittest.TestCase):
    '''Class to test OS maps data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(osReader.numRecords, 2151)

        for fieldName in osReader.data:
            self.assertEqual(osReader.data[fieldName].size, 2151)


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(osReader.data['lat'].min(), np.float64(50.7221081))
        self.assertEqual(osReader.data['lat'].max(), np.float64(50.7359201))


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(osReader.data['lon'].min(), np.float64(-2.8814909))
        self.assertEqual(osReader.data['lon'].max(), np.float64(-2.8223704))

In [8]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    filename = os.path.join(projdir, 'sessions', '20220411', 'APEX_Pro_Speedsurfing20220411111317.gpx')
    corosReader = GpxReader(filename)

    filename = os.path.join(projdir, 'sessions', '20220411', 'GT31_1Hz_GEORG30MICHA_932000175_20220411_111600.gpx')
    sbnReader = GpxReader(filename)

    filename = os.path.join(projdir, 'sessions', '20220411', 'GT31_1Hz_GEORG30MICHA_932000175_20220512_094254_DLG.gpx')
    sbpReader = GpxReader(filename)

    filename = os.path.join(projdir, 'sessions', '20220422', 'activity_8686414511.gpx')
    fenix6Reader = GpxReader(filename)

    filename = os.path.join(projdir, 'sessions', 'misc', 'Seatownandgoldencap.gpx')
    osReader = GpxReader(filename)

    pc1 = time.perf_counter()
    corosReader.load()
    sbnReader.load()
    sbpReader.load()
    fenix6Reader.load()
    osReader.load()
    pc2 = time.perf_counter()

    print("\nGPX files loaded in in %0.2f seconds" % (pc2 - pc1))


GPX files loaded in in 0.85 seconds


In [9]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

.................................
----------------------------------------------------------------------
Ran 33 tests in 0.016s

OK
